In [1]:
import itertools
import math
import re
from pathlib import Path

import geopandas as gp
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
def replace_region(name):
    try:
        name = name.replace("MIMAROPA REGION", "REGION IV-B (MIMAROPA)")
        name = name.replace("REGION III - CENTRAL LUZON", "REGION III (Central Luzon)")
        name = name.replace("REGION II - CAGAYAN VALLEY", "REGION II (Cagayan Valley)")
        name = name.replace(
            "REGION VIII - EASTERN VISAYAS", "REGION VIII (Eastern Visayas)"
        )
        name = name.replace("REGION I - ILOCOS", "REGION I (Ilocos Region)")
        name = name.replace("REGION IV-A - CALABARZON", "REGION IV-A (CALABARZON)")
        name = name.replace(
            "CORDILLERA ADMINISTRATIVE REGION", "CAR - Cordillera Administrative Region"
        )
        name = name.replace(
            "REGION VI - WESTERN VISAYAS", "REGION VI (Western Visayas)"
        )
        name = name.replace(
            "AUTONOMOUS REGION IN MUSLIM MINDANAO",
            "BARMM - Bangsamoro Autonomous Region in Muslim Mindanao",
        )
        name = name.replace("REGION XII - SOCCSKSARGEN", "REGION XII (Soccsksargen)")
        name = name.replace(
            "REGION VII - CENTRAL VISAYAS", "REGION VII (Central Visayas)"
        )
        name = name.replace("REGION XIII - CARAGA", "REGION XIII (Caraga)")
        name = name.replace(
            "REGION IX - ZAMBOANGA PENINSULA", "REGION IX (Zamboanga Peninsula)"
        )
        name = name.replace(
            "REGION X - NORTHERN MINDANAO", "REGION X (Northern Mindanao)"
        )
        name = name.replace("REGION V - BICOL", "REGION V (Bicol Region)")
        name = name.replace("REGION XI - DAVAO", "REGION XI (Davao Region)")
        name = name.replace("NATIONAL CAPITAL REGION", "NCR - National Capital Region")
        return name
    except:
        return name

In [3]:
# Setting up directories

WORKINGDIR = Path(os.getcwd())
PROJECTROOT = WORKINGDIR.parents[1]

DATASET = Path(
    PROJECTROOT,
    "data",
    "gathered-datasets",
    "housing-census",
    "housing-census-water-supply-cooking-2015.csv",
)

REG_PROV = Path(
    PROJECTROOT,
    "data",
    "gathered-datasets",
    "housing-census",
    "region-province.csv",
)

DATASET_DEST = Path(
    PROJECTROOT,
    "data",
    "gathered-datasets",
    "housing-census",
    "housing-census-water-supply-cooking-2015-flattened.csv",
)

REF_DF = Path(PROJECTROOT, "data", "cleaned-datasets", "ph-shp-file", "ph-shp-file.shp")

In [4]:
def strip_names(city):
    try:
        return city.strip()
    except AttributeError:
        return np.nan

In [5]:
df = pd.read_csv(DATASET)
df.head(20)

City/Municipality Number of Households*  \
0                       NaN                   NaN   
1   NATIONAL CAPITAL REGION            3.095.484    
2       METROPOLITAN MANILA            3.095.484    
3            CITY OF MANILA              435.154    
4       CITY OF MANDALUYONG              100.356    
5          CITY OF MARIKINA               98.238    
6             CITY OF PASIG              180.612    
7               QUEZON CITY              683.044    
8          CITY OF SAN JUAN               28.623    
9             CALOOCAN CITY              367.878    
10          CITY OF MALABON               86.191    
11          CITY OF NAVOTAS               60.904    
12       CITY OF VALENZUELA              153.041    
13        CITY OF LAS PIÑAS              141.925    
14           CITY OF MAKATI              154.095    
15       CITY OF MUNTINLUPA              122.286    
16        CITY OF PARAÑAQUE              163.074    
17               PASAY CITY              107.619    
18                  PATEROS               14.188    
19              TAGUIG CITY              198.256    

   Own use faucet community water system Shared faucet community water system  \
0                                    NaN                                  NaN   
1                             2.469.267                              440.905    
2                             2.469.267                              440.905    
3                               347.935                               69.597    
4                                89.749                                7.551    
5                                89.537                                6.991    
6                               162.226                               13.063    
7                               561.729                              102.919    
8                                24.496                                3.008    
9                               283.142                               66.524    
10                               62.428                               19.561    
11                               42.529                               13.190    
12                              111.294                               31.136    
13                              111.950                               11.997    
14                              132.241                               13.107    
15                               70.351                                6.366    
16                              126.870                               24.213    
17                               87.737                               19.367    
18                               12.740                                1.062    
19                              152.313                               31.253    

   Own use tubed/piped deep well Shared tubed/piped deep well  \
0                            NaN                          NaN   
1                        21.969                       34.255    
2                        21.969                       34.255    
3                         1.181                        1.488    
4                           107                          254    
5                           290                          389    
6                           197                          435    
7                         2.375                        3.006    
8                           -                            -      
9                         5.172                        5.595    
10                          412                          416    
11                           73                          367    
12                        2.056                        3.234    
13                          617                        1.869    
14                          258                          473    
15                        7.553                       13.422    
16                          397                        1.510    
17                      

In [6]:
first_col = "City/Municipality"	

In [7]:
df[first_col] = df[first_col].apply(strip_names)

In [8]:
# Group into regions
df["group"] = df.isnull().all(axis=1).cumsum()

In [9]:
regions = list(df["group"].unique())
regions

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [10]:
df.head(12)

City/Municipality Number of Households*  \
0                       NaN                   NaN   
1   NATIONAL CAPITAL REGION            3.095.484    
2       METROPOLITAN MANILA            3.095.484    
3            CITY OF MANILA              435.154    
4       CITY OF MANDALUYONG              100.356    
5          CITY OF MARIKINA               98.238    
6             CITY OF PASIG              180.612    
7               QUEZON CITY              683.044    
8          CITY OF SAN JUAN               28.623    
9             CALOOCAN CITY              367.878    
10          CITY OF MALABON               86.191    
11          CITY OF NAVOTAS               60.904    

   Own use faucet community water system Shared faucet community water system  \
0                                    NaN                                  NaN   
1                             2.469.267                              440.905    
2                             2.469.267                              440.905    
3                               347.935                               69.597    
4                                89.749                                7.551    
5                                89.537                                6.991    
6                               162.226                               13.063    
7                               561.729                              102.919    
8                                24.496                                3.008    
9                               283.142                               66.524    
10                               62.428                               19.561    
11                               42.529                               13.190    

   Own use tubed/piped deep well Shared tubed/piped deep well  \
0                            NaN                          NaN   
1                        21.969                       34.255    
2                        21.969                       34.255    
3                         1.181                        1.488    
4                           107                          254    
5                           290                          389    
6                           197                          435    
7                         2.375                        3.006    
8                           -                            -      
9                         5.172                        5.595    
10                          412                          416    
11                           73                          367    

   Tubed/piped shallow well  Dug well Protected spring Unprotected spring  \
0                       NaN       NaN              NaN                NaN   
1                    1.704     2.157            1.337                 27    
2                    1.704     2.157            1.337                 27    
3                      100        48              233                  9    
4                       55         1               36                  2    
5                       14        16               91                -      
6                       94        34               43                -      
7                      334       343              293                  4    
8                      -         -                -                  -      
9                      191       290               15                -      
10                      18        11              -                  -      
11                      28         3               60                -      

   Lake river rain and others    Peddler Bottled water    Others Not Reported  \
0                         NaN        NaN           NaN       NaN          NaN   
1                         35     40.197        80.566     3.065          -      
2                         35     40.197        80.566     3.065          -      
3                         13      9.313         5.068       169          -      
4            

In [11]:
df[first_col] = df[
    first_col
].apply(replace_region)

In [12]:
water_supply = df.columns[2:15].tolist()
water_supply

['Own use faucet community water system',
 'Shared faucet community water system',
 'Own use tubed/piped deep well',
 'Shared tubed/piped deep well',
 'Tubed/piped shallow well',
 'Dug well',
 'Protected spring',
 'Unprotected spring',
 'Lake river rain and others',
 'Peddler',
 'Bottled water',
 'Others',
 'Not Reported']

In [13]:
rp_df = pd.read_csv(REG_PROV)
rp_df.head()

region                          province
0           NCR - National Capital Region               METROPOLITAN MANILA
1  CAR - Cordillera Administrative Region                              ABRA
2  CAR - Cordillera Administrative Region   BENGUET (excluding Baguio City)
3  CAR - Cordillera Administrative Region                            IFUGAO
4  CAR - Cordillera Administrative Region                           KALINGA

In [14]:
rp_df["province"] = rp_df["province"].apply(strip_names)

In [15]:
all_vals = []

In [16]:
def parse_region(df):
    df = df.copy()
    region_name = df[first_col].loc[1]
    unique_vals = df[first_col].dropna().unique()

    provinces = list(rp_df["province"].loc[rp_df["region"]==region_name].unique()) 
    
    df["province_no"] = df[first_col].isin(provinces).cumsum()
    df = df.loc[df["province_no"] != 0]
    
    province_no_list = df["province_no"].unique()
    
    for province in province_no_list:
            province_df = df.loc[df["province_no"]==province]
            parse_province(province_df, region_name)

In [17]:
def parse_province(province_df, region_name):
    province_df = province_df.copy()
    province_df = province_df.reset_index()
    
    province_name = province_df[first_col].iloc[0]
    unique_vals = province_df[first_col].dropna().unique()
    
    cities = [val for val in unique_vals if val not in [region_name, province_name, "Not Reported"]]
    
    for city in cities:
        print(f">>> Parsing {city},{province_name} in {region_name}...")
        idx = (
            province_df.loc[province_df[first_col] == city].index
        ).tolist()[0]
        
        parse_city(city.strip(), region_name, province_df, idx, province_name)

In [18]:
def parse_city(city, region_name, province_df, idx, province_name):
    city_df = province_df[idx:idx+10]
    
    city_vals = {}
    city_vals["city"] = city
    city_vals["region_name"] = region_name
    city_vals["province"] = province_name

    for supply in water_supply:        
        try:
            val = city_df[supply].loc[city_df[first_col] == city]
            val = str(val.values[0]).strip()
            val = val.replace(".", "")
        except IndexError:
            continue

        try:
            city_vals[f"{supply.strip()}_count"] = float(val)
        except:
            city_vals[f"{supply.strip()}_count"] = float(np.nan)
                
        all_vals.append(city_vals)

In [19]:
df.head()

City/Municipality Number of Households*  \
0                            NaN                   NaN   
1  NCR - National Capital Region            3.095.484    
2            METROPOLITAN MANILA            3.095.484    
3                 CITY OF MANILA              435.154    
4            CITY OF MANDALUYONG              100.356    

  Own use faucet community water system Shared faucet community water system  \
0                                   NaN                                  NaN   
1                            2.469.267                              440.905    
2                            2.469.267                              440.905    
3                              347.935                               69.597    
4                               89.749                                7.551    

  Own use tubed/piped deep well Shared tubed/piped deep well  \
0                           NaN                          NaN   
1                       21.969                       34.255    
2                       21.969                       34.255    
3                        1.181                        1.488    
4                          107                          254    

  Tubed/piped shallow well  Dug well Protected spring Unprotected spring  \
0                      NaN       NaN              NaN                NaN   
1                   1.704     2.157            1.337                 27    
2                   1.704     2.157            1.337                 27    
3                     100        48              233                  9    
4                      55         1               36                  2    

  Lake river rain and others    Peddler Bottled water    Others Not Reported  \
0                        NaN        NaN           NaN       NaN          NaN   
1                        35     40.197        80.566     3.065          -      
2                        35     40.197        80.566     3.065          -      
3                        13      9.313         5.068       169          -      
4                       -          875         1.637        89          -      

   group  
0      1  
1      1  
2      1  
3      1  
4      1

In [20]:
for region in regions:
    df_ = df.loc[df["group"] == region].reset_index()
    region = df_.iloc[1][first_col]
    parse_region(df_)

>>> Parsing CITY OF MANILA,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF MANDALUYONG,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF MARIKINA,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF PASIG,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing QUEZON CITY,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF SAN JUAN,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CALOOCAN CITY,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF MALABON,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF NAVOTAS,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF VALENZUELA,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF LAS PIÑAS,METROPOLITAN MANILA in NCR - National Capital Region...
>>> Parsing CITY OF MAKATI,METROPOLITAN MANILA in NCR - National Capital R

In [21]:
final_df = pd.DataFrame.from_dict(all_vals).drop_duplicates()
final_df

city                                        region_name  \
0           CITY OF MANILA                      NCR - National Capital Region   
13     CITY OF MANDALUYONG                      NCR - National Capital Region   
26        CITY OF MARIKINA                      NCR - National Capital Region   
39           CITY OF PASIG                      NCR - National Capital Region   
52             QUEZON CITY                      NCR - National Capital Region   
...                    ...                                                ...   
19474             TANDUBAS  BARMM - Bangsamoro Autonomous Region in Muslim...   
19487       TURTLE ISLANDS  BARMM - Bangsamoro Autonomous Region in Muslim...   
19500             LANGUYAN  BARMM - Bangsamoro Autonomous Region in Muslim...   
19513            SAPA-SAPA  BARMM - Bangsamoro Autonomous Region in Muslim...   
19526               SIBUTU  BARMM - Bangsamoro Autonomous Region in Muslim...   

                  province  Own use faucet community water system_count  \
0      METROPOLITAN MANILA                                     347935.0   
13     METROPOLITAN MANILA                                      89749.0   
26     METROPOLITAN MANILA                                      89537.0   
39     METROPOLITAN MANILA                                     162226.0   
52     METROPOLITAN MANILA                                     561729.0   
...                    ...                                          ...   
19474            TAWI-TAWI                                        344.0   
19487            TAWI-TAWI                                          3.0   
19500            TAWI-TAWI                                       1060.0   
19513            TAWI-TAWI                                         14.0   
19526            TAWI-TAWI                                        259.0   

       Shared faucet community water system_count  \
0                                         69597.0   
13                                         7551.0   
26                                         6991.0   
39                                        13063.0   
52                                       102919.0   
...                                           ...   
19474                                      1551.0   
19487                                         NaN   
19500                                       262.0   
19513                                        15.0   
19526                                      1034.0   

       Own use tubed/piped deep well_count  \
0                                   1181.0   
13                                   107.0   
26                                   290.0   
39                                   197.0   
52                                  2375.0   
...                                    ...   
19474                                  6.0   
19487                                  1.0   
19500                                 71.0   
19513                                115.0   
19526                                 28.0   

       Shared tubed/piped deep well_count  Tubed/piped shallow well_count  \
0                                  1488.0                           100.0   
13                                  254.0                            55.0   
26                                  389.0                            14.0   
39                                  435.0                            94.0   
52                                 3006.0                           334.0   
...                                   ...                             ...   
19474                                 7.0                             9.0   
19487                                20.0                             2.0   
19500                                56.0                            89.0   
19513                               819.0                             5.0   
19526                               693.0                           655.0   

       Dug well_count  Protected 

In [22]:
final_df.head(40)

city                             region_name  \
0          CITY OF MANILA           NCR - National Capital Region   
13    CITY OF MANDALUYONG           NCR - National Capital Region   
26       CITY OF MARIKINA           NCR - National Capital Region   
39          CITY OF PASIG           NCR - National Capital Region   
52            QUEZON CITY           NCR - National Capital Region   
65       CITY OF SAN JUAN           NCR - National Capital Region   
78          CALOOCAN CITY           NCR - National Capital Region   
91        CITY OF MALABON           NCR - National Capital Region   
104       CITY OF NAVOTAS           NCR - National Capital Region   
117    CITY OF VALENZUELA           NCR - National Capital Region   
130     CITY OF LAS PIÑAS           NCR - National Capital Region   
143        CITY OF MAKATI           NCR - National Capital Region   
156    CITY OF MUNTINLUPA           NCR - National Capital Region   
169     CITY OF PARAÑAQUE           NCR - National Capital Region   
182            PASAY CITY           NCR - National Capital Region   
195               PATEROS           NCR - National Capital Region   
208           TAGUIG CITY           NCR - National Capital Region   
221     BANGUED (Capital)  CAR - Cordillera Administrative Region   
234               BOLINEY  CAR - Cordillera Administrative Region   
247                 BUCAY  CAR - Cordillera Administrative Region   
260                BUCLOC  CAR - Cordillera Administrative Region   
273             DAGUIOMAN  CAR - Cordillera Administrative Region   
286               DANGLAS  CAR - Cordillera Administrative Region   
299               DOLORES  CAR - Cordillera Administrative Region   
312                LA PAZ  CAR - Cordillera Administrative Region   
325                 LACUB  CAR - Cordillera Administrative Region   
338           LAGANGILANG  CAR - Cordillera Administrative Region   
351               LAGAYAN  CAR - Cordillera Administrative Region   
364              LANGIDEN  CAR - Cordillera Administrative Region   
377  LICUAN-BAAY (LICUAN)  CAR - Cordillera Administrative Region   
390                  LUBA  CAR - Cordillera Administrative Region   
403             MALIBCONG  CAR - Cordillera Administrative Region   
416                MANABO  CAR - Cordillera Administrative Region   
429            PEÑARRUBIA  CAR - Cordillera Administrative Region   
442               PIDIGAN  CAR - Cordillera Administrative Region   
455                 PILAR  CAR - Cordillera Administrative Region   
468            SALLAPADAN  CAR - Cordillera Administrative Region   
481            SAN ISIDRO  CAR - Cordillera Administrative Region   
494              SAN JUAN  CAR - Cordillera Administrative Region   
507           SAN QUINTIN  CAR - Cordillera Administrative Region   

                province  Own use faucet community water system_count  \
0    METROPOLITAN MANILA                                     347935.0   
13   METROPOLITAN MANILA                                      89749.0   
26   METROPOLITAN MANILA                                      89537.0   
39   METROPOLITAN MANILA                                     162226.0   
52   METROPOLITAN MANILA                                     561729.0   
65   METROPOLITAN MANILA                                      24496.0   
78   METROPOLITAN MANILA                                     283142.0   
91   METROPOLITAN MANILA                                      62428.0   
104  METROPOLITAN MANILA                                      42529.0   
117  METROPOLITAN MANILA                                     111294.0   
130  METROPOLITAN MANILA                                     111950.0   
143  METROPOLITAN MANILA                                     132241.0   
156  METROPOLITAN MANILA                                      70351.0   
169  METROPOLITAN MANILA                                     126870.0   
182  METROPOLITAN MANILA                                      87737.0   
195  METROPOLITAN MANIL

In [23]:
final_df["year"]=2015
final_df.head(10)

city                    region_name             province  \
0         CITY OF MANILA  NCR - National Capital Region  METROPOLITAN MANILA   
13   CITY OF MANDALUYONG  NCR - National Capital Region  METROPOLITAN MANILA   
26      CITY OF MARIKINA  NCR - National Capital Region  METROPOLITAN MANILA   
39         CITY OF PASIG  NCR - National Capital Region  METROPOLITAN MANILA   
52           QUEZON CITY  NCR - National Capital Region  METROPOLITAN MANILA   
65      CITY OF SAN JUAN  NCR - National Capital Region  METROPOLITAN MANILA   
78         CALOOCAN CITY  NCR - National Capital Region  METROPOLITAN MANILA   
91       CITY OF MALABON  NCR - National Capital Region  METROPOLITAN MANILA   
104      CITY OF NAVOTAS  NCR - National Capital Region  METROPOLITAN MANILA   
117   CITY OF VALENZUELA  NCR - National Capital Region  METROPOLITAN MANILA   

     Own use faucet community water system_count  \
0                                       347935.0   
13                                       89749.0   
26                                       89537.0   
39                                      162226.0   
52                                      561729.0   
65                                       24496.0   
78                                      283142.0   
91                                       62428.0   
104                                      42529.0   
117                                     111294.0   

     Shared faucet community water system_count  \
0                                       69597.0   
13                                       7551.0   
26                                       6991.0   
39                                      13063.0   
52                                     102919.0   
65                                       3008.0   
78                                      66524.0   
91                                      19561.0   
104                                     13190.0   
117                                     31136.0   

     Own use tubed/piped deep well_count  Shared tubed/piped deep well_count  \
0                                 1181.0                              1488.0   
13                                 107.0                               254.0   
26                                 290.0                               389.0   
39                                 197.0                               435.0   
52                                2375.0                              3006.0   
65                                   NaN                                 NaN   
78                                5172.0                              5595.0   
91                                 412.0                               416.0   
104                                 73.0                               367.0   
117                               2056.0                              3234.0   

     Tubed/piped shallow well_count  Dug well_count  Protected spring_count  \
0                             100.0            48.0                   233.0   
13                             55.0             1.0                    36.0   
26                             14.0            16.0                    91.0   
39                             94.0            34.0                    43.0   
52                            334.0           343.0                   293.0   
65                              NaN             NaN                     NaN   
78                            191.0           290.0                    15.0   
91                             18.0            11.0                     NaN   
104                            28.0             3.0                    60.0   
117                            33.0           309.0                     NaN   

     Unprotected spring_count  Lake river rain and others_count  \
0                         9.0                              13.0   
13                        2.0                               NaN   
26                        NaN                       

In [24]:
final_df.to_csv(DATASET_DEST)